In [8]:
import boto3
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "DataSource_PostgresDB_Credentials"
    region_name = "eu-west-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    secret = get_secret_value_response['SecretString']
    return secret

In [9]:
# Test the function
try:
    result = get_secret()
    print("Secret retrieved successfully")
    # Be careful not to print the actual secret in a production environment
except Exception as e:
    print(f"Error retrieving secret: {str(e)}")

Secret retrieved successfully


In [12]:
import boto3
from botocore.exceptions import ClientError
from pg8000.native import Connection, Error
import pandas as pd
import json

def get_secret():
    secret_name = "DataSource_PostgresDB_Credentials"
    region_name = "eu-west-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    return secret

def create_connection():
    # Retrieve the secret
    secret = get_secret()
    secret_dict = json.loads(secret)

    # Extract credentials from the secret
    user = secret_dict['user']
    password = secret_dict['password']
    host = secret_dict['host']
    database = secret_dict['database']
    port = secret_dict['port']

    # Establish the database connection
    conn = Connection(
        user=user,
        database=database,
        host=host,
        password=password,
        port=port
    )
    return conn

In [25]:
conn = create_connection()

try:
    query = 'SELECT * FROM sales_order LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_sales_order = pd.DataFrame(results, columns=column_names)
    print("Table: sales_order")
    print(df_sales_order.to_string(index=False))
    # print(df_sales_order)
finally:
    conn.close()

Table: sales_order
 sales_order_id              created_at            last_updated  design_id  staff_id  counterparty_id  units_sold unit_price  currency_id agreed_delivery_date agreed_payment_date  agreed_delivery_location_id
              2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          3        19                8       42972       3.94            2           2022-11-07          2022-11-08                            8
              3 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4        10                4       65839       2.91            3           2022-11-06          2022-11-07                           19
              4 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4        10               16       32069       3.89            2           2022-11-05          2022-11-07                           15
              5 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          7        18                4       49659       2.41            3           2

In [14]:
conn = create_connection()

try:
    query = 'SELECT * FROM design LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_design = pd.DataFrame(results, columns=column_names)
    print("Table: design")
    print(df_design.to_string(index=False))
finally:
    conn.close()

Table: design
 design_id              created_at design_name file_location                   file_name            last_updated
         8 2022-11-03 14:20:49.962      Wooden          /usr   wooden-20220717-npgz.json 2022-11-03 14:20:49.962
        51 2023-01-12 18:50:09.935      Bronze      /private   bronze-20221024-4dds.json 2023-01-12 18:50:09.935
        69 2023-02-07 17:31:10.093      Bronze   /lost+found   bronze-20230102-r904.json 2023-02-07 17:31:10.093
        16 2022-11-22 15:02:10.226        Soft       /System     soft-20211001-cjaz.json 2022-11-22 15:02:10.226
        54 2023-01-16 09:14:09.775     Plastic    /usr/ports  plastic-20221206-bw3l.json 2023-01-16 09:14:09.775
        55 2023-01-19 08:10:10.138    Concrete  /opt/include concrete-20210614-04nd.json 2023-01-19 08:10:10.138
        10 2022-11-03 14:20:49.962        Soft    /usr/share     soft-20220201-hzz1.json 2022-11-03 14:20:49.962
        57 2023-01-19 10:37:09.965      Cotton /etc/periodic   cotton-20220527-vn4

In [15]:
conn = create_connection()

try:
    query = 'SELECT * FROM currency LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_currency = pd.DataFrame(results, columns=column_names)
    print("Table: currency")
    print(df_currency.to_string(index=False))
finally:
    conn.close()

Table: currency
 currency_id currency_code              created_at            last_updated
           1           GBP 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
           2           USD 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
           3           EUR 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962


In [16]:
conn = create_connection()

try:
    query = 'SELECT * FROM staff LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_staff = pd.DataFrame(results, columns=column_names)
    print("Table: staff")
    print(df_staff.to_string(index=False))
finally:
    conn.close()

Table: staff
 staff_id first_name last_name  department_id                       email_address              created_at            last_updated
        1    Jeremie    Franey              2    jeremie.franey@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        2      Deron     Beier              6       deron.beier@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        3   Jeanette    Erdman              6   jeanette.erdman@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        4        Ana    Glover              3        ana.glover@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        5  Magdalena     Zieme              8   magdalena.zieme@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        6      Korey   Kreiger              3     korey.kreiger@terrifictotes.com 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
        7    Raphael    Rippin              2    raphael.rippin@terrifictotes

In [17]:
conn = create_connection()

try:
    query = 'SELECT * FROM counterparty LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_counterparty = pd.DataFrame(results, columns=column_names)
    print("Table: counterparty")
    print(df_counterparty.to_string(index=False))
finally:
    conn.close()

Table: counterparty
 counterparty_id        counterparty_legal_name  legal_address_id commercial_contact        delivery_contact              created_at            last_updated
               1                 Fahey and Sons                15        Micheal Toy Mrs. Lucy Runolfsdottir 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
               2    Leannon, Predovic and Morar                28      Melba Sanford           Jean Hane III 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
               3                  Armstrong Inc                 2          Jane Wiza            Myra Kovacek 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
               4                     Kohler Inc                29        Taylor Haag       Alfredo Cassin II 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
               5    Frami, Yundt and Macejkovic                22     Homer Mitchell         Ivan Balistreri 2022-11-03 14:20:51.563 2022-11-03 14:20:51.563
               6                      

In [18]:
conn = create_connection()

try:
    query = 'SELECT * FROM address LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_address = pd.DataFrame(results, columns=column_names)
    print("Table: address")
    print(df_address.to_string(index=False))
finally:
    conn.close()

Table: address
 address_id       address_line_1   address_line_2        district              city postal_code                               country       phone              created_at            last_updated
          1      6826 Herzog Via             None            Avon New Patienceburgh       28441                                Turkey 1803 637401 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
          2    179 Alexie Cliffs             None            None       Aliso Viejo  99305-7380                            San Marino 9621 880720 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
          3     148 Sincere Fort             None            None      Lake Charles       89360                                 Samoa 0730 783349 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
          4   6102 Rogahn Skyway             None    Bedfordshire         Olsonside       47518                     Republic of Korea 1239 706295 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
          5    

In [19]:
conn = create_connection()

try:
    query = 'SELECT * FROM department LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_department = pd.DataFrame(results, columns=column_names)
    print("Table: department")
    print(df_department.to_string(index=False))
finally:
    conn.close()

Table: department
 department_id department_name   location        manager              created_at            last_updated
             1           Sales Manchester   Richard Roma 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             2      Purchasing Manchester Naomi Lapaglia 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             3      Production      Leeds   Chester Ming 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             4        Dispatch       Leds     Mark Hanna 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             5         Finance Manchester Jordan Belfort 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             6      Facilities Manchester Shelley Levene 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             7  Communications      Leeds      Ann Blake 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
             8              HR      Leeds     James Link 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962


In [20]:
conn = create_connection()

try:
    query = 'SELECT * FROM purchase_order LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_purchase_order = pd.DataFrame(results, columns=column_names)
    print("Table: purchase_order")
    print(df_purchase_order.to_string(index=False))
finally:
    conn.close()

Table: purchase_order
 purchase_order_id              created_at            last_updated  staff_id  counterparty_id item_code  item_quantity item_unit_price  currency_id agreed_delivery_date agreed_payment_date  agreed_delivery_location_id
                 1 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187        12               11   ZDOI5EA            371          361.39            2           2022-11-09          2022-11-07                            6
                 2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186        20               17   QLZLEXR            286          199.04            2           2022-11-04          2022-11-07                            8
                 3 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187        12               15   AN3D85L            839          658.58            2           2022-11-05          2022-11-04                           16
                 5 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186        18                2   I9MET53   

In [21]:
conn = create_connection()

try:
    query = 'SELECT * FROM payment_type LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_payment_type = pd.DataFrame(results, columns=column_names)
    print("Table: payment_type")
    print(df_payment_type.to_string(index=False))
finally:
    conn.close()

Table: payment_type
 payment_type_id payment_type_name              created_at            last_updated
               1     SALES_RECEIPT 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
               2      SALES_REFUND 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
               3  PURCHASE_PAYMENT 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962
               4   PURCHASE_REFUND 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962


In [22]:
conn = create_connection()

try:
    query = 'SELECT * FROM payment LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_payment = pd.DataFrame(results, columns=column_names)
    print("Table: payment")
    print(df_payment.to_string(index=False))
finally:
    conn.close()

Table: payment
 payment_id              created_at            last_updated  transaction_id  counterparty_id payment_amount  currency_id  payment_type_id  paid payment_date  company_ac_number  counterparty_ac_number
          2 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187               2               15      552548.62            2                3 False   2022-11-04           67305075                31622269
          3 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186               3               18      205952.22            3                1 False   2022-11-03           81718079                47839086
          5 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187               5               17       57067.20            2                3 False   2022-11-06           66213052                91659548
          8 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186               8                2      254007.12            3                3 False   2022-11-05           32948439          

In [23]:
conn = create_connection()

try:
    query = 'SELECT * FROM transaction LIMIT 10;'
    results = conn.run(query)

    column_names = [desc['name'] for desc in conn.columns]
    df_transaction = pd.DataFrame(results, columns=column_names)
    print("Table: transaction")
    print(df_transaction.to_string(index=False))
finally:
    conn.close()

Table: transaction
 transaction_id transaction_type  sales_order_id  purchase_order_id              created_at            last_updated
              1         PURCHASE             NaN                2.0 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186
              2         PURCHASE             NaN                3.0 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187
              3             SALE             1.0                NaN 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186
              4         PURCHASE             NaN                1.0 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187
              5         PURCHASE             NaN                4.0 2022-11-03 14:20:52.187 2022-11-03 14:20:52.187
              6             SALE             2.0                NaN 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186
              7             SALE             3.0                NaN 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188
              8         PURCHASE             NaN     